# Visualizing CASP Data

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from skbio.alignment import global_pairwise_align_protein
from skbio import Protein

In [10]:
def get_tertiary_matrix(tertiary, i):
    x = tertiary[0][3 * i], tertiary[0][3 * i + 1], tertiary[0][3 * i + 2]
    y = tertiary[1][3 * i], tertiary[1][3 * i + 1], tertiary[1][3 * i + 2]
    z = tertiary[2][3 * i], tertiary[2][3 * i + 1], tertiary[2][3 * i + 2]
    return x, y, z

In [11]:
id = "2B6G_2_A"

primary = Protein("GSNVFNNTITHPNAGPTSATSTSTSSNGNTPLSSNSSMNPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY")

partial_primary = Protein("NPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY")
secondary = " HHHHS HHHHH HHHHHHHHT HHHHHHHTTS HHHHTT  HHHHHHHS   HHHHHHHHHHHHHHHHHHHHT S GGG "

In [12]:
result = global_pairwise_align_protein(primary, partial_primary)

/home/nishaq/Documents/research/PTSP/.ptsp/lib/python3.6/site-packages/skbio/alignment/_pairwise.py:599: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  "to track progress on this).", EfficiencyWarning)


In [17]:
print(result[0][0])

GSNVFNNTITHPNAGPTSATSTSTSSNGNTPLSSNSSMNPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY


In [18]:
print(result[0][1])

--------------------------------------NPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY


In [26]:
aligned_primary = str(result[0][1])
new_secondary = []

In [27]:
i = 0
for aa in aligned_primary:
    if aa == '-':
        new_secondary.append('C')
    elif secondary[i] == ' ':
        new_secondary.append('C')
        i += 1
    else:
        new_secondary.append(secondary[i])
        i += 1

In [28]:
print(''.join(new_secondary))

CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHSCHHHHHCHHHHHHHHTCHHHHHHHTTSCHHHHTTCCHHHHHHHSCCCHHHHHHHHHHHHHHHHHHHHTCSCGGGC


In [29]:
print(result[0][0])

GSNVFNNTITHPNAGPTSATSTSTSSNGNTPLSSNSSMNPKSLTDPKLLKNIPMWLKSLRLHKYSDALSGTPWIELIYLDDETLEKKGVLALGARRKLLKAFGIVIDYKERDLIDRSAY
